<h1>Real-time OPR for scouting</h1>
<h3>Created for <a href="http://www.firstinspires.org/robotics/frc">First Robotics Competition</a> (FRC) <a href="http://kauaibots.com/">Team 2465: Kauai'Bots</a></h3>
<p>Calculates (an improved) OPR in real-time from the FRC Event API (<a href="http://docs.frcevents2.apiary.io/">documentation</a>, <a href="https://usfirst.collab.net/sf/projects/first_community_developers/">project home</a>)</p>
<br><p>Copyright 2016 Andrew Dyami Cadwallader</p>

Acquire API credentials by joining the<a href="https://usfirst.collab.net/sf/projects/first_community_developers/"> FRC Event API project.</a>

In [1]:
import urllib.request
import base64
import json
import time
import numpy as np
import pandas as pd
import getpass

<h3>Format API request</h3>

In [2]:
usr = getpass.getpass('FRC Event API project Username: ')

FRC Event API project Username: ········


In [3]:
pwd = getpass.getpass('FRC Event API project AuthorizationKey: ')

FRC Event API project AuthorizationKey: ········


In [4]:
def pull(username, AuthorizationKey, url):
    Token = base64.b64encode(str.encode(username+':'+AuthorizationKey)).decode("utf-8")
    headers = {'Accept' : 'Application/JSON', 'Authorization' : 'Basic '+Token}
    request = urllib.request.Request(url, None, headers)
    with urllib.request.urlopen(request) as response:
        return json.loads(response.read().decode("utf-8")) 
        #response.read() outputs binary
        #.decode("utf-8") converts binary to string
        #json.loads(str) converts json formatted string to dictionary object

<h3>Retrieve and reformat teams and alliances for all <a href="http://docs.frcevents2.apiary.io/#reference/match-results/event-match-results/retrieve-event-match-results">matches</a> at an event</h3>

In [5]:
def pullteams(event):
    url = 'https://frc-api.firstinspires.org/v2.0/2016/matches/'+event
    return pull(usr,pwd,url) #returns 

In [6]:
def teamtable(json, matchtype='Qualification'):    
    t_table = [[team['teamNumber'] for team in match['Teams']] for match in json['Matches'] 
               if match['tournamentLevel'] == matchtype]
    r1, r2, r3, b1, b2, b3 = zip(*t_table)
    reds = list(zip(r1,r2,r3))
    blues = list(zip(b1,b2,b3))
    t_table = reds + blues
    return t_table

In [7]:
def teamlist(table):
    #create list of unique team numbers
    return list(sorted(set([team for alliance in table for team in alliance])))

In [8]:
def teammatrix(table, _list):
    #columns are team numbers, rows are matches, entry (r,c) is 1 or 0 indicating whether team participated in match 
    return np.matrix([[1 if team in alliance else 0 for team in _list] for alliance in table])

In [9]:
def processteams(event):
    t_json = pullteams(event)
    t_table = teamtable(t_json)
    t_list = teamlist(t_table)
    t_matrix = teammatrix(t_table, t_list)
    return t_matrix, t_list

<h3>Retrieve and reformat <a href="http://docs.frcevents2.apiary.io/#reference/match-results/score-details/retrieve-event-score-details">score details</a> for all matches at an event</h3>

In [10]:
def pullscores(event):
    url='https://frc-api.firstinspires.org/v2.0/2016/scores/'+event+'/Qualification'
    return pull(usr,pwd,url)

In [11]:
def scorearray(json): #Returns list of lists, columns are score categories, rows are matches, reds on top
    #return [[match['Alliances'][alliance][scorecategory] for scorecategory in sorted(match['Alliances'][alliance])] 
            #for alliance in (1,0) for match in json['MatchScores']]
    red=[[match['Alliances'][1][score] for score in sorted(match['Alliances'][1])] for match in json['MatchScores']]
    blue=[[match['Alliances'][0][score] for score in sorted(match['Alliances'][0])] for match in json['MatchScores']]
    return red+blue

In [12]:
def scorematrix(table):
    table = list(zip(*table)) #transpose, columns become matches, rows become score categories
    table = [list(table[row]) for row in (3,4,5,7,11,26,27,28,29,30,32,33,34)] #score categories in "scorelabels"
    table[0]=(np.array(table[0])*10).tolist() #autoBouldersHigh
    table[1]=(np.array(table[1])*5).tolist() #autoBouldersLow
    table[5]=(np.array(table[5])*5).tolist() #teleopBouldersHigh
    table[6]=(np.array(table[6])*2).tolist() #teleopBouldersLow
    table[12]=(np.array(table[12])-np.array(table[4])).tolist() #totalPoints - foulPoints, deassign foulpoints
    table[4]=np.roll(np.array(table[4])*-1,int(len(table[4])/2)).tolist() #swap team for foulPoints
    table[12]=(np.array(table[12])+np.array(table[4])).tolist() #reassign foulPoints to committing team
    table[9]=[int(value)*20 for value in table[9]] #change Boolean to int for teleopDefensesBreached
    table[11]=[int(value)*25 for value in table[11]] #change Boolean to int for teleopTowerCaptured
    return np.matrix(list(zip(*table))) #transpose again and cast to matrix

In [13]:
def scorelabels():
    return ['autoBouldersHigh','autoBouldersLow','autoCrossingPoints','autoReachPoints','foulPoints',
            'teleopBouldersHigh','teleopBouldersLow','teleopChallengePoints','teleopCrossingPoints',
            'teleopDefensesBreached','teleopScalePoints','teleopTowerCaptured','totalPoints']

In [14]:
def processscores(event):
    return scorematrix(scorearray(pullscores(event)))

<h3>Put it all together</h3>

In [15]:
def realtimeOPR(event):
    labels = scorelabels()
    t_matrix, t_list = processteams(event)
    scores = processscores(event)
    OPRs = t_matrix.I*scores
    return pd.DataFrame(OPRs, t_list, labels)

In [17]:
HIHO = realtimeOPR('HIHO')

In [19]:
HIHO.sort_values('teleopCrossingPoints',ascending=False)

,autoBouldersHigh,autoBouldersLow,autoCrossingPoints,autoReachPoints,foulPoints,teleopBouldersHigh,teleopBouldersLow,teleopChallengePoints,teleopCrossingPoints,teleopDefensesBreached,teleopScalePoints,teleopTowerCaptured,totalPoints
2465,0.723553,-0.071929,4.387863,1.320536,0.602001,-1.403205,-0.608971,4.681889,20.386846,11.521141,-0.825113,-0.690978,29.193470
2439,0.170167,0.416583,9.892240,-0.029431,1.128920,1.535752,6.322557,4.900207,16.593383,10.784440,0.617517,3.856309,41.547895
2504,0.108693,0.023640,1.911551,1.261841,0.682729,0.083732,0.434596,3.253828,16.225181,6.777650,-0.273257,-0.018985,23.712534
3132,-0.184523,0.015630,7.527538,0.450399,-0.524213,2.105205,0.296309,5.351725,13.479233,12.675253,-0.985584,-0.570465,27.531718
2445,-0.346190,-0.077006,4.927838,1.055344,1.071123,-2.228811,2.064454,3.960959,13.412388,6.856209,1.827663,-1.548388,25.667763
3878,-0.890146,0.001395,0.167399,0.969926,-2.966481,0.567714,0.268528,3.416724,13.098434,5.436484,0.323835,2.124966,14.957327
3880,-1.143008,-0.021904,11.471176,-0.105358,-1.166304,0.034018,6.905470,3.520150,12.929139,7.919505,-0.302030,1.447657,32.121349
4218,-1.694271,0.026718,-2.217013,0.970807,-1.195037,0.654771,1.010147,2.789141,12.352948,2.492214,-0.631250,-0.723836,12.066960
359,2.395788,0.422907,12.781812,-0.074237,0.491692,20.600561,0.354697,5.626842,12.216658,12.517403,0.787066,5.698444,55.603785
2437,1.231989,-0.019741,3.758273,0.645582,-2.098477,3.120629,-0.512289,4.452157,11.430719,3.865569,-0.311170,0.910396,21.697671
